In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tkinter import *

In [2]:
class channel:
    """The channel class represents the physical voltage inputs for the SLIM instrument.
    The channel class represents a MIPS input."""
    
    def __init__(self, description, voltage):
        self.description = description
        self.voltage = voltage
    
    def set_voltage(self, new_voltage):
        self.voltage = new_voltage

class region:
    """The region class represents the space between two voltage inputs. 
    Over a defined length, the two voltage inputs create the separation field."""
    def __init__(self, description, length, field_strength, rungs=None):
        self.description = description
        self.length = length
        self.field_strength = field_strength
        self.rungs=rungs
        
    def set_length(self, new_length):
        self.length = new_length
        
    def set_field_strength(self, new_field_strength):
        self.field_strength = new_field_strength
        
    def voltage_drop(self):
        voltage_drop = self.field_strength * self.length
        return voltage_drop
    
    def voltage_drop_per_rung(self):
        if self.rungs == None:
            raise ValueError('Number of rungs undefined.')
        else:
            voltage_drop_per_rung = self.voltage_drop() / self.rungs
            return voltage_drop_per_rung

In [82]:
# Define the region and channels of the CIF
CIF = region('CIF', 10.87, 0)
CSSO = channel('CCSO', 0)
CIF_CL_to_CCSO_drop = 0
CIF_CL = channel('CIFCL', 0)
CIF_to_CIF_CL_drop = 0
CIF_dc_out = channel('CIF_dc_out', 0)
CIF_dc_in = channel('CIF_dc_in', 0)

# Define the region and channels of SLIM 2
SLIM2 = region('SLIM2', 38.1, 0, rungs=250)
SLIM2_dc_in = channel('SLIM2_dc_in', 0)
SLIM2_dc_out = channel('SLIM2_dc_out', 0)
M8_sg = channel('Module 8 switch guard', 0)
SLIM2_to_CIF_drop = 0
M8_sg_bias = 0

#Define channels in the interface region
M8_orth_out = channel('M8_orth_out', 0)
M8_orth_in = channel('M8_orth_in', 0)
M8_orth = region('M8_orth', 3.5052, 0)
SLIM2_8M_interface = 0
M8_orth_to_SLIM2 = 0

#Define channels in the interface region
M7_orth_out = channel('M7_orth_out', 0)
M7_orth_in = channel('M7_orth_in', 0)
M7_orth = region('M7_orth', 3.5052, 3.75)
SLIM1_7M_interface = 0

# #Define drops in the region
M7_to_M8_drop = 0

# #Define channels and regions
# SLIM1_dc_out = channel('SLIM1_dc_out', 0)
# SLIM1_dc_in = channel('SLIM1_dc_in', 0)
# M7_sg = channel('Module 7 switch guard', 0)
# SLIM1 = region('SLIM1', 45.72, 4, rungs=300)

# #Define interface and voltage drops
# SLIM1_to_7M_orth_drop = 0
# M7_sg_bias = 5

# Application Window

In [70]:
window = Tk()
window.title("SLIM voltage control")
window.geometry('1000x1000')

#CIF Region
ccso_lbl = Label(window, text="Collision Cell Static Offset", font=("Arial Bold", 10))
ccso_lbl.grid(column=0, row=0)
ccso_entry = Entry(window, width=5)
ccso_entry.grid(column=1, row=0)

CIF_to_CSSO_drop_lbl = Label(window, text="CIF CL to CSSO drop", font=("Arial Bold", 10))
CIF_to_CSSO_drop_lbl.grid(column=0, row=1)
CIF_to_CSSO_drop_entry = Entry(window, width=5)
CIF_to_CSSO_drop_entry.grid(column=1, row=1)

CIF_to_CIF_CL_drop_lbl = Label(window, text="CIF to CIF CL drop", font=("Arial Bold", 10))
CIF_to_CIF_CL_drop_lbl.grid(column=0, row=2)
CIF_to_CIF_CL_drop_entry = Entry(window, width=5)
CIF_to_CIF_CL_drop_entry.grid(column=1, row=2)

CIF_fs_lbl = Label(window, text="CIF field strength", font=("Arial Bold", 10))
CIF_fs_lbl.grid(column=0, row=3)
CIF_fs_entry = Entry(window, width=5)
CIF_fs_entry.grid(column=1, row=3)

#SLIM2 Region
SLIM2_to_CIF_drop_lbl = Label(window, text="SLIM2 to CIF drop", font=("Arial Bold", 10))
SLIM2_to_CIF_drop_lbl.grid(column=0, row=4)
SLIM2_to_CIF_drop_entry = Entry(window, width=5)
SLIM2_to_CIF_drop_entry.grid(column=1,row=4)

SLIM2_fs_lbl = Label(window, text="SLIM2 Field Strength", font=("Arial Bold", 10))
SLIM2_fs_lbl.grid(column=0, row=5)
SLIM2_fs_entry = Entry(window, width=5)
SLIM2_fs_entry.grid(column=1, row=5)

#Orthogonal Region
M8_sg_bias_lbl = Label(window, text="8M Switch Guard Bias", font=("Arial Bold", 10))
M8_sg_bias_lbl.grid(column=0, row=6)
M8_sg_bias_entry = Entry(window, width=5)
M8_sg_bias_entry.grid(column=1, row=6)

M8_orth_fs_lbl = Label(window, text="8M Orth Field Strength", font=("Arial Bold", 10))
M8_orth_fs_lbl.grid(column=0, row=7)
M8_orth_fs_entry = Entry(window, width=5)
M8_orth_fs_entry.grid(column=1, row=7)

M8_orth_to_SLIM2_lbl = Label(window, text="8M Orth Field Strength", font=("Arial Bold", 10))
M8_orth_to_SLIM2_lbl.grid(column=0, row=8)
M8_orth_to_SLIM2_entry = Entry(window, width=5)
M8_orth_to_SLIM2_entry.grid(column=1, row=8)

M7_to_M8_drop_lbl = Label(window, text="7M to 8M voltage drop"), font=("Arial Bold", 10))
M7_to_M8_drop_lbl.grid(column=0, row=9)
M7_to_M8_drop_entry = Entry(window, width=5)
M7_to_M8_drop_entry.grid(column=1, row=9)

def update_voltage_settings():
    CSSO.set_voltage(int(ccso_entry.get()))
    global CIF_CL_to_CCSO_drop
    CIF_CL_to_CCSO_drop = int(CIF_to_CSSO_drop_entry.get())
    global CIF_to_CIF_CL_drop
    CIF_to_CIF_CL_drop = int(CIF_to_CIF_CL_drop_entry.get())
    CIF.set_field_strength(int(CIF_fs_entry.get()))
    global SLIM2_to_CIF_drop
    SLIM2_to_CIF_drop = int(SLIM2_to_CIF_drop_entry.get())
    SLIM2.set_field_strength(int(SLIM2_fs_entry.get()))
    global M8_sg_bias
    M8_sg_bias = int(M8_sg_bias_entry.get())
    M8_orth.set_field_strength(int(M8_orth_fs_entry.get()))
    global M8_orth_to_SLIM2
    M8_orth_to_SLIM2 = int(M8_orth_to_SLIM2_entry.get())
    global M7_to_M8_drop
    M7_to_M8_drop = 

update_voltages_btn = Button(window, text="Update voltages", command=update_voltage_settings)
update_voltages_btn.grid(column=2, row=0)

window.mainloop()

In [71]:
CSSO.voltage

2

In [72]:
CIF_CL_to_CCSO_drop

2

In [73]:
CIF.field_strength

2

In [74]:
CIF_to_CIF_CL_drop

2

In [75]:
SLIM2_to_CIF_drop

2

In [76]:
SLIM2.field_strength

2

In [77]:
M8_sg_bias

2

In [78]:
M8_orth.field_strength

2